In [52]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import os
from urllib.request import urlopen, quote,URLError, HTTPError
import time

In [9]:
data = pd.read_excel(r"G:\work\logistica\test\1812zhejiang.xlsx")

In [10]:
data.columns

Index(['订单编号', '订单类型', '订单状态', '订单子状态', '取消原因', '订单挂起原因', '订单来源', '店铺', 'AB类型',
       '订单金额', '订单生成时间', '支付完成时间', '交易完成时间', '发货时间', '入网用户姓名', '入网手机号',
       '入网身份证号', '联系人电话号码', '收货人姓名', '收货地址所在省 ', ' 收货地址市 ', ' 收货地址县', '收货地址',
       '收货人电话号码', '所在省 / 市 / 县', '入网人所在省/市/县', '号码归属地', '配送方式', 'CPS推荐人',
       '订单备注', '物流单号', '物流签收时间', '承运商', '写卡渠道', '销售品编号', '销售品名称', '销售品类型',
       '套餐', '主号码', '副号码', '靓号等级', '靓号低消', '靓号预存款', '营业厅送货方式', '是否线下模式',
       '是否线下转线上', '线下转线上原因', '营业厅送货iccid', '发货量', '签收量', '激活量', '月拦截量', '所属省',
       '所属市', '产品分类', '激活时效', '性别', '签收时效', '交付时效', '年龄', '配送时效', '收货地址经度',
       '收货地址纬度', '网点名称', '网点地址', '网点经度', '网点纬度', '网点距离'],
      dtype='object')

In [26]:
df = data.copy()

In [16]:
# df['所在省 / 市 / 县'] = df['所在省 / 市 / 县'].map(lambda x:str(x))
# split = pd.DataFrame((x.split('/') for x in df['所在省 / 市 / 县']), index=df.index, columns=['所在省','所在市','所在县'])
# df = pd.merge(df,split,left_index=True,right_index=True)

In [30]:
df_ZJ = df[df["收货地址所在省 "] == "浙江"]

In [34]:
df_ZJ['收货地址所在省 '] = df_ZJ['收货地址所在省 '].map(lambda x:str(x))
df_ZJ[ ' 收货地址市 '] = df_ZJ[ ' 收货地址市 '] .map(lambda x:str(x))
df_ZJ['收货地址'] = df_ZJ['收货地址'].map(lambda x:str(x))

d:\python\envs\jupyterpy36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
d:\python\envs\jupyterpy36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
d:\python\envs\jupyterpy36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://p

In [35]:
add = df_ZJ['收货地址所在省 '] + df_ZJ[ ' 收货地址市 '] + df_ZJ['收货地址']

In [36]:
df_ZJ["承运商"].value_counts()

EMS     11285
京东物流     6100
京东线下     3760
韵达       1166
Name: 承运商, dtype: int64

In [44]:
JD_df_ZJ = df_ZJ[df_ZJ["承运商"].str.contains("韵达|EMS") == False]

In [45]:
JD_df_ZJ["承运商"].value_counts()

京东物流    6100
京东线下    3760
Name: 承运商, dtype: int64

In [49]:
ak_gaode = 'c71d9eda293d20db64955275557d92d4'

In [46]:
def get_json_gaode(address,ak):
    ak = "c71d9eda293d20db64955275557d92d4"
    url="http://restapi.amap.com/v3/geocode/geo?key=%s&address=%s"%(ak,address)
    data=requests.get(url)
    contest=data.json()
    return contest

In [61]:
def get_JD_siteInfo_gaode(address):
    print("开始获取高德地址信息")
    JD_siteList_gaode = []
    for b in address:
        JD_siteDict_gaode = {}
        try:
            temp=get_json_gaode(b,ak_gaode)
        except HTTPError as e:
            print("请求出错")
            pass
        else:
            if ('geocodes' in temp):
                JD_siteDict_gaode["JD_siteLocation_gaode"] = temp['geocodes'][0]['location']
                JD_siteDict_gaode["JD_siteFormatted_address_gaode"] = temp['geocodes'][0]['formatted_address']
                JD_siteDict_gaode["JD_siteProvince_gaode"] = temp['geocodes'][0]['province']
                JD_siteDict_gaode["JD_siteCountry_gaode"] = temp['geocodes'][0]['country']
                if ('citycode' in temp):
                    JD_siteDict_gaode["JD_siteCitycode_gaode"] = temp['geocodes'][0]['citycode']
                else:
                    pass
                JD_siteDict_gaode["JD_siteCity_gaode"] = temp['geocodes'][0]['city']
                JD_siteDict_gaode["JD_siteDistrict_gaode"] = temp['geocodes'][0]['district']
                JD_siteDict_gaode["JD_siteLevel_gaode"] = temp['geocodes'][0]['level']
                JD_siteDict_gaode["JD_siteTownship_gaode"] = temp['geocodes'][0]['township']
                JD_siteDict_gaode["JD_siteAdcode_gaode"] = temp['geocodes'][0]['adcode']
                JD_siteDict_gaode["JD_siteStreet_gaode"] = temp['geocodes'][0]['street']
                JD_siteDict_gaode["JD_siteNumber_gaode"] = temp['geocodes'][0]['number']
            else:
                pass
        JD_siteList_gaode.append(JD_siteDict_gaode)
    return JD_siteList_gaode

In [62]:
df_JD_siteList_gaode = pd.DataFrame(get_JD_siteInfo_gaode(add))

开始获取高德地址信息


ConnectionError: HTTPConnectionPool(host='restapi.amap.com', port=80): Max retries exceeded with url: /v3/geocode/geo?key=c71d9eda293d20db64955275557d92d4&address=%E6%B5%99%E6%B1%9F%E6%9D%AD%E5%B7%9E%E5%B8%82%E4%BC%A0%E5%AA%92%E5%AD%A6%E9%99%A2%E7%94%9F%E6%B4%BB%E8%B4%B913%E5%8F%B7%E6%A5%BC (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000000013BD9780>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。',))

In [60]:
df_JD_siteList_gaode

,JD_siteAdcode_gaode,JD_siteCity_gaode,JD_siteCountry_gaode,JD_siteDistrict_gaode,JD_siteFormatted_address_gaode,JD_siteLevel_gaode,JD_siteLocation_gaode,JD_siteNumber_gaode,JD_siteProvince_gaode,JD_siteStreet_gaode,JD_siteTownship_gaode
0,330604,绍兴市,中国,上虞区,浙江省绍兴市上虞区望江路,道路,"120.693934,30.132474",[],浙江省,[],[]
1,330681,绍兴市,中国,诸暨市,浙江省绍兴市诸暨市草塔,兴趣点,"120.145154,29.689000",[],浙江省,[],[]
2,330402,嘉兴市,中国,南湖区,浙江省嘉兴市南湖区嘉兴第四高级中学,兴趣点,"120.791254,30.732320",[],浙江省,[],[]
3,330302,温州市,中国,鹿城区,浙江省温州市鹿城区美琪鞋业,兴趣点,"120.576609,28.039658",[],浙江省,[],[]
4,330503,湖州市,中国,南浔区,浙江省湖州市南浔区浔东幼儿园,兴趣点,"120.438961,30.878947",[],浙江省,[],[]
5,331004,台州市,中国,路桥区,浙江省台州市路桥区路南街道,乡镇,"121.397120,28.558600",[],浙江省,[],[]
6,331081,台州市,中国,温岭市,浙江省台州市温岭市水澄村,村庄,"121.338110,28.483078",[],浙江省,[],[]
7,330483,嘉兴市,中国,桐乡市,浙江省嘉兴市桐乡市环北西区|13幢,门牌号,"120.545313,30.644538",[],浙江省,[],[]
8,330104,杭州市,中国,江干区,浙江省杭州市江干区菜鸟驿站,兴趣点,"120.360597,30.315942",[],浙江省,[],[]
9,330212,宁波市,中国,鄞州区,浙江省宁波市鄞州区格兰云天,兴趣点,"121.556945,29.825879",[],浙江省,[],[]
